In [5]:
import datetime as dt
import pandas as pd
import os
import itertools
import functions.common as cf
from scripts import interactive_inputs
from scripts import data_request_urls_ooi1_0
from scripts import send_data_requests_nc
from scripts import data_request_tools

In [15]:
sDir = '/Users/leila/Documents/Desktop/8thEGOMeeting-notebooks'
f = '/Users/leila/Documents/NSFEduSupport/github/8thEGOMeeting-notebooks/instrument_list.csv'
username = 'OOIAPI-4J3Y5ISGDUPDB7'
token = '4OT8YQUME0XOQF'

cf.create_dir(sDir)
now = dt.datetime.now().strftime('%Y%m%dT%H%M')

In [16]:
df = pd.read_csv(os.path.join(sDir, f))

In [17]:
url_list = []
for i, j in df.iterrows():
    array = data_request_tools.check_str(j['array'])
    array = data_request_tools.format_inputs(array)
    refdes = j['reference_designator']
    if type(refdes) == str:
        subsite = data_request_tools.format_inputs(refdes.split('-')[0])
        node = data_request_tools.format_inputs(refdes.split('-')[1])
        inst = data_request_tools.format_inputs('-'.join((refdes.split('-')[2], refdes.split('-')[3])))
    else:
        subsite = data_request_tools.check_str(j['subsite'])
        subsite = data_request_tools.format_inputs(subsite)
        node = data_request_tools.check_str(j['node'])
        node = data_request_tools.format_inputs(node)
        inst = data_request_tools.check_str(j['sensor'])
        inst = data_request_tools.format_inputs(inst)
    delivery_methods = data_request_tools.check_str(j['delivery_method'])
    delivery_methods = data_request_tools.format_inputs(delivery_methods)
    
    print(refdes, array, subsite, node, inst, delivery_methods)
    
    urls = data_request_urls_ooi1_0.main(sDir, array, subsite, node, inst, delivery_methods, now)
    url_list.append(urls)

CP05MOAS-GL336-03-CTDGVM000 ['CP'] ['CP05MOAS'] ['GL336'] ['03-CTDGVM000'] ['telemetered']
CP05MOAS-GL336-04-DOSTAM000 ['CP'] ['CP05MOAS'] ['GL336'] ['04-DOSTAM000'] ['telemetered']
CP05MOAS-GL336-05-PARADM000 ['CP'] ['CP05MOAS'] ['GL336'] ['05-PARADM000'] ['telemetered']
CP05MOAS-GL336-02-FLORTM000 ['CP'] ['CP05MOAS'] ['GL336'] ['02-FLORTM000'] ['telemetered']
CP05MOAS-GL336-03-CTDGVM000 ['CP'] ['CP05MOAS'] ['GL336'] ['03-CTDGVM000'] ['recovered_host']


Exception: Invalid delivery_method: recovered_host

In [ ]:
f_url_list = list(itertools.chain(*url_list))

thredds_output_urls = send_data_requests_nc.main(sDir, f_url_list, username, token, now)

print('\nSeeing if the requests have fulfilled...')
for i in range(len(thredds_output_urls)):
    url = thredds_output_urls[i]
    print('\nDataset {} of {}: {}'.format((i + 1), len(thredds_output_urls), url))
    if 'no_output_url' not in url:
        cf.check_request_status(url)